In [1]:
# -*- coding: utf-8 -*-
# %%
import argparse
import pandas as pd
import glob
import datetime
import matplotlib.pyplot as plt
import act
import os
import sys
import xarray as xr
from tempfile import TemporaryDirectory

import plotly.express as px 
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import cufflinks as cf
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import plotly.io as pio

In [2]:
# %%
USERNAME = os.getenv("ARM_USERNAME")
TOKEN = os.getenv("ARM_TOKEN")
SAIL_DATA_STREAM = 'gucdlppiM1.b1'
DATA_STREAM_FILEEXT = '.cdf'
DATE_FORMAT = "%Y-%m-%d"

In [3]:
# %%
def create_dl_plots(startdate):
    enddate = (
        datetime.datetime.strptime(startdate, DATE_FORMAT) + datetime.timedelta(days=2)
    ).strftime(DATE_FORMAT)

    with TemporaryDirectory() as temp_dir:
        act.discovery.download_data(USERNAME, TOKEN, SAIL_DATA_STREAM, startdate, enddate, output=temp_dir)
        ppi_files = glob.glob(''.join([temp_dir, '/', SAIL_DATA_STREAM, '*'+DATA_STREAM_FILEEXT]))
        print(len(ppi_files))
        ppi_ds = act.io.armfiles.read_netcdf(ppi_files)
    
        ppi_ds['time_hour_and_min']  = ppi_ds.time.to_series().apply(lambda dt: dt.replace(second=0, microsecond=0))

        objs = []
        wind_obj=None
        # Split ppi dataset into chunks with full scans
        # and calculate the winds for each gucdlppi dataset.
        for key, group in ppi_ds.groupby("time_hour_and_min"):
            wind_obj = act.retrievals.compute_winds_from_ppi(
                group, 
                # remove_all_missing=True, 
                # snr_threshold=0.008
            )
            objs.append(wind_obj)
        wind_obj = xr.merge(objs)

    src_prof = wind_obj.resample(time='60Min').median().to_dataframe().reset_index()
    src_prof = src_prof.reset_index().set_index('time').tz_localize("UTC").tz_convert("US/Mountain").tz_localize(None).reset_index()
    # get data for a complete, local time, day
    src_prof = src_prof[
        src_prof['time'].dt.day == datetime.datetime.strptime(startdate, DATE_FORMAT).day
    ]

    src_prof = src_prof.query('height <= 2035')
    
    src_prof['day_hour'] = src_prof['time'].dt.strftime('%D %H')
    src_prof['minute'] = src_prof['time'].dt.minute
    src_prof['hour'] = src_prof['time'].dt.hour

    src_prof['hour_group'] = pd.cut(
        src_prof['hour'],
        4,
        labels=['0-5', '6-11', '12-17', '18-23']
    )
    return src_prof


In [4]:
windy_days = ['2022-12-02','2022-12-13', '2022-12-14', '2022-12-16', '2022-12-22',
               '2022-12-26', '2023-01-04', '2023-01-18', '2023-01-25',
               '2023-02-06', '2023-02-09', '2023-02-19', '2023-02-22',
               '2023-03-16', '2023-03-17', '2023-03-27', '2023-03-30',
               '2023-03-31']
all_days = pd.date_range(start='2022-12-01', end='2023-03-31', freq='D').strftime('%Y-%m-%d')
dfs = []

Index(['2022-12-01', '2022-12-02', '2022-12-03', '2022-12-04', '2022-12-05',
       '2022-12-06', '2022-12-07', '2022-12-08', '2022-12-09', '2022-12-10',
       ...
       '2023-03-22', '2023-03-23', '2023-03-24', '2023-03-25', '2023-03-26',
       '2023-03-27', '2023-03-28', '2023-03-29', '2023-03-30', '2023-03-31'],
      dtype='object', length=121)

In [5]:
create=True
if create is True:
    for day in all_days:
        dfs.append(create_dl_plots(startdate=day))
else:
    files = glob.glob('./01_data/processed_data/sail/*.csv')
    dfs = []
    for file in files:
        dfs.append(pd.read_csv(file))
    df = pd.concat(dfs)

/tmp/ipykernel_28315/1534367022.py:8: DeprecationWarning:

act.discovery.get_armfiles.download_data will be retired in version 2.0.0.  Please use act.discovery.arm.download_arm_data instead.



[DOWNLOADING] gucdlppiM1.b1.20221201.131502.cdf
[DOWNLOADING] gucdlppiM1.b1.20221202.203003.cdf
[DOWNLOADING] gucdlppiM1.b1.20221202.054502.cdf
[DOWNLOADING] gucdlppiM1.b1.20221202.081502.cdf
[DOWNLOADING] gucdlppiM1.b1.20221201.224502.cdf
[DOWNLOADING] gucdlppiM1.b1.20221201.163003.cdf
[DOWNLOADING] gucdlppiM1.b1.20221202.134502.cdf
[DOWNLOADING] gucdlppiM1.b1.20221202.124503.cdf
[DOWNLOADING] gucdlppiM1.b1.20221201.123002.cdf
[DOWNLOADING] gucdlppiM1.b1.20221201.150021.cdf
[DOWNLOADING] gucdlppiM1.b1.20221202.061502.cdf
[DOWNLOADING] gucdlppiM1.b1.20221201.001503.cdf
[DOWNLOADING] gucdlppiM1.b1.20221202.121502.cdf
[DOWNLOADING] gucdlppiM1.b1.20221202.000021.cdf
[DOWNLOADING] gucdlppiM1.b1.20221201.003002.cdf
[DOWNLOADING] gucdlppiM1.b1.20221202.011502.cdf
[DOWNLOADING] gucdlppiM1.b1.20221202.213002.cdf
[DOWNLOADING] gucdlppiM1.b1.20221201.143003.cdf
[DOWNLOADING] gucdlppiM1.b1.20221202.013003.cdf
[DOWNLOADING] gucdlppiM1.b1.20221201.171503.cdf
[DOWNLOADING] gucdlppiM1.b1.20221201.184

/tmp/ipykernel_28315/1534367022.py:11: DeprecationWarning:

act.io.armfiles.read_netcdf will be replaced in version 2.0.0 by act.io.arm.read_arm_netcdf()

/tmp/ipykernel_28315/1534367022.py:8: DeprecationWarning:

act.discovery.get_armfiles.download_data will be retired in version 2.0.0.  Please use act.discovery.arm.download_arm_data instead.



[DOWNLOADING] gucdlppiM1.b1.20221203.213002.cdf
[DOWNLOADING] gucdlppiM1.b1.20221202.203003.cdf
[DOWNLOADING] gucdlppiM1.b1.20221203.084502.cdf
[DOWNLOADING] gucdlppiM1.b1.20221202.054502.cdf
[DOWNLOADING] gucdlppiM1.b1.20221203.113002.cdf
[DOWNLOADING] gucdlppiM1.b1.20221203.064503.cdf
[DOWNLOADING] gucdlppiM1.b1.20221202.081502.cdf
[DOWNLOADING] gucdlppiM1.b1.20221203.123003.cdf
[DOWNLOADING] gucdlppiM1.b1.20221203.174502.cdf
[DOWNLOADING] gucdlppiM1.b1.20221203.120021.cdf
[DOWNLOADING] gucdlppiM1.b1.20221203.204503.cdf
[DOWNLOADING] gucdlppiM1.b1.20221203.201503.cdf
[DOWNLOADING] gucdlppiM1.b1.20221203.183003.cdf
[DOWNLOADING] gucdlppiM1.b1.20221202.134502.cdf
[DOWNLOADING] gucdlppiM1.b1.20221202.124503.cdf
[DOWNLOADING] gucdlppiM1.b1.20221203.114503.cdf
[DOWNLOADING] gucdlppiM1.b1.20221203.093002.cdf
[DOWNLOADING] gucdlppiM1.b1.20221202.061502.cdf
[DOWNLOADING] gucdlppiM1.b1.20221202.121502.cdf
[DOWNLOADING] gucdlppiM1.b1.20221203.223003.cdf
[DOWNLOADING] gucdlppiM1.b1.20221202.000

/tmp/ipykernel_28315/1534367022.py:11: DeprecationWarning:

act.io.armfiles.read_netcdf will be replaced in version 2.0.0 by act.io.arm.read_arm_netcdf()

/tmp/ipykernel_28315/1534367022.py:8: DeprecationWarning:

act.discovery.get_armfiles.download_data will be retired in version 2.0.0.  Please use act.discovery.arm.download_arm_data instead.



...

/tmp/ipykernel_28315/1534367022.py:11: DeprecationWarning:

act.io.armfiles.read_netcdf will be replaced in version 2.0.0 by act.io.arm.read_arm_netcdf()

/tmp/ipykernel_28315/1534367022.py:8: DeprecationWarning:

act.discovery.get_armfiles.download_data will be retired in version 2.0.0.  Please use act.discovery.arm.download_arm_data instead.



[DOWNLOADING] gucdlppiM1.b1.20221225.134503.cdf
[DOWNLOADING] gucdlppiM1.b1.20221226.060021.cdf
[DOWNLOADING] gucdlppiM1.b1.20221226.054502.cdf
[DOWNLOADING] gucdlppiM1.b1.20221226.073002.cdf
[DOWNLOADING] gucdlppiM1.b1.20221226.233002.cdf
[DOWNLOADING] gucdlppiM1.b1.20221225.153003.cdf
[DOWNLOADING] gucdlppiM1.b1.20221226.183003.cdf
[DOWNLOADING] gucdlppiM1.b1.20221226.234503.cdf
[DOWNLOADING] gucdlppiM1.b1.20221225.033003.cdf
[DOWNLOADING] gucdlppiM1.b1.20221225.040022.cdf
[DOWNLOADING] gucdlppiM1.b1.20221225.130021.cdf
[DOWNLOADING] gucdlppiM1.b1.20221225.224503.cdf
[DOWNLOADING] gucdlppiM1.b1.20221226.050021.cdf
[DOWNLOADING] gucdlppiM1.b1.20221225.200023.cdf
[DOWNLOADING] gucdlppiM1.b1.20221225.110022.cdf
[DOWNLOADING] gucdlppiM1.b1.20221225.084503.cdf
[DOWNLOADING] gucdlppiM1.b1.20221226.013003.cdf
[DOWNLOADING] gucdlppiM1.b1.20221225.011503.cdf
[DOWNLOADING] gucdlppiM1.b1.20221225.173003.cdf
[DOWNLOADING] gucdlppiM1.b1.20221225.064503.cdf
[DOWNLOADING] gucdlppiM1.b1.20221226.040

/tmp/ipykernel_28315/1534367022.py:11: DeprecationWarning:

act.io.armfiles.read_netcdf will be replaced in version 2.0.0 by act.io.arm.read_arm_netcdf()

/tmp/ipykernel_28315/1534367022.py:8: DeprecationWarning:

act.discovery.get_armfiles.download_data will be retired in version 2.0.0.  Please use act.discovery.arm.download_arm_data instead.



[DOWNLOADING] gucdlppiM1.b1.20221227.094503.cdf
[DOWNLOADING] gucdlppiM1.b1.20221226.060021.cdf
[DOWNLOADING] gucdlppiM1.b1.20221227.190021.cdf
[DOWNLOADING] gucdlppiM1.b1.20221226.054502.cdf
[DOWNLOADING] gucdlppiM1.b1.20221226.073002.cdf
[DOWNLOADING] gucdlppiM1.b1.20221226.233002.cdf
[DOWNLOADING] gucdlppiM1.b1.20221226.183003.cdf
[DOWNLOADING] gucdlppiM1.b1.20221227.221503.cdf
[DOWNLOADING] gucdlppiM1.b1.20221226.234503.cdf
[DOWNLOADING] gucdlppiM1.b1.20221227.124503.cdf
[DOWNLOADING] gucdlppiM1.b1.20221227.110022.cdf


KeyboardInterrupt: 

In [6]:
# filter out times when wind speed error or wind direction error are high
df = df[(df['wind_direction_error']<90) & (df['wind_speed_error']<5) & (df['height']>50)]

In [8]:
colors = px.colors.qualitative.Plotly
for i,hour_group in enumerate(df.hour_group.unique()):
    # make it a two panel plot
    fig = make_subplots(rows=1, cols=2, shared_yaxes=True, shared_xaxes=False, subplot_titles=(f'Wind Speed', 'Wind Direction'))

    df_hr = df.query(f'hour_group == "{hour_group}"')
    for i, hr in enumerate(df_hr.hour.unique()):
        if i == 0:
            showscale = True
        else:
            showscale = False
        fig.add_trace(go.Scatter(
            x=df_hr.query(f'hour == {hr}')['wind_speed'],
            y=df_hr.query(f'hour == {hr}')['height'],
            mode='markers',
            marker=dict(
                size=12,
                color=colors[i],
                line=dict(width=2, color='DarkSlateGrey')
            ),
            name=str(hr)
        ), 
        row=1, col=1)    

        fig.add_trace(go.Scatter(
            x=df_hr.query(f'hour == {hr}')['wind_direction'],
            y=df_hr.query(f'hour == {hr}')['height'],
            mode='markers',
            marker=dict(
                size=12,
                color=colors[i],
                line=dict(width=2, color='DarkSlateGrey')
            ),
            showlegend=False,
            name=str(hr)
        ), 
        row=1, col=2, )    
        fig.update_traces(hovertemplate='Height: %{y}m<br>Wind Speed: %{x}m/s')
    fig.update_layout(
        title='Wind Speed and Direction',
        xaxis_title='Wind Speed (m/s)',
        yaxis_title='Height (m)',
        legend_title='Hour Group',
        height=600,
        width=800
    )
    # move the legend to the top
    fig.update_layout(legend=dict(
        yanchor="top",
        y=0.99,
        xanchor="left",
        x=0.01,
    ))
    fig.update_xaxes(range=[0, 25],
                     tickvals=[0, 5, 10, 15, 20, 25],row=1, col=1)
    # update the hover to show the hour
    

In [19]:
# Pull in Kettle Ponds data
kettle_ponds_hourly = xr.open_dataset("./01_data/processed_data/sos/sos_ds_1H_max_wspd_storage.nc")
kettle_ponds_hourly_10m_winds = kettle_ponds_hourly[['spd_10m_c','dir_10m_c']]
# set time zone to UTC
kettle_ponds_hourly_10m_winds['time'] = pd.to_datetime(kettle_ponds_hourly_10m_winds.time.values).tz_localize('UTC').tz_convert('US/Mountain').tz_localize(None)

In [20]:
df['time'] = pd.to_datetime(df['time'])
high_vals = df[(df['height']>690) & (df['height']<705)]
high_vals.index = high_vals['time']
# convert to datetime
high_vals.index = pd.to_datetime(high_vals.index)

# select all the same times as high vals
kettle_ponds_hourly_10m_winds = kettle_ponds_hourly_10m_winds.to_dataframe()

In [21]:
kettle_ponds_hourly_10m_winds

,spd_10m_c,dir_10m_c
time,,
2022-11-30 17:00:00,2.740854,324.577881
2022-11-30 18:00:00,3.190895,317.789917
2022-11-30 19:00:00,3.845659,317.611298
2022-11-30 20:00:00,2.591963,326.194061
2022-11-30 21:00:00,3.315059,319.800690
...,...,...
2023-03-31 13:00:00,8.403108,325.327362
2023-03-31 14:00:00,6.571191,296.683075
2023-03-31 15:00:00,6.995218,317.516083


In [22]:
# rename wind speed and direction by adding high and low to the column names for wind_speed and wind_direction
high_vals = high_vals.rename(columns={'wind_speed':'high_wind_speed', 'wind_direction':'high_wind_direction'})

df_combined = pd.concat([high_vals[['high_wind_speed', 'high_wind_direction']], kettle_ponds_hourly_10m_winds[['spd_10m_c', 'dir_10m_c']]], axis=1,).dropna()
# save df_combined 
if not os.path.exists('./01_data/processed_data/windy_days_dl_winds_700m_and_kp_10m_winds.csv'):
    df_combined.to_csv('./01_data/processed_data/windy_days_dl_winds_700m_and_kp_10m_winds.csv')

In [24]:
fig = go.Figure()
fig.add_trace(go.Scatter(
    x=df_combined['spd_10m_c'],
    y=df_combined['high_wind_speed'],
    mode='markers',
    marker=dict(
        size=12,
        # color by time of day
        color=df_combined['dir_10m_c'],
        # add colorscale
        colorscale='twilight_r',
        colorbar=dict(
                    #   title='Surface wind<br>direction',
                      title_font=dict(size=24),
                      tickfont=dict(size=24)),
                      # change ticks to N, NW, W, SW, S, SE, E, NE
                        colorbar_tickvals=[0, 45, 90, 135, 180, 225, 270, 315],
                        colorbar_ticktext=['N', 'NE', 'E', 'SE', 'S', 'SW', 'W', 'NW'],
        showscale=True,
        line=dict(width=2, color='DarkSlateGrey'),
        
    ),
    customdata=df_combined['dir_10m_c'],
    hovertemplate=('<b>Surface Winds: %{x:.2f} m/s <br>Winds Aloft: %{y:.2} m/s</b><BR>Direction:<BR>' +
                           '%{customdata}' +
                           '<extra></extra>'),
    
))
fig.update_layout(
    # title='Wind Speed Aloft vs. Near Surface on Windy Days',
    xaxis_title='Winds at 10 m above surface (m/s)',
    yaxis_title='Winds at 700 m above surface (m/s)',
    height=800,
    width=900,
    xaxis=dict(title_font=dict(size=24),),
    yaxis=dict(title_font=dict(size=24),),
    paper_bgcolor='rgba(0,0,0,0)',
)

# increase the ytick font size
fig.update_yaxes(tickfont=dict(size=24),
                 range=[0,25])
fig.update_xaxes(tickfont=dict(size=24),
                 range=[0,25])
# save with a transparent background
# fig.write_image("./04_products/figures/sandbox/surface_vs_aloft_winds_synoptic.png", scale=10)


In [25]:
fig = go.Figure()
fig.add_trace(go.Scatter(
    x=df_combined['high_wind_speed'],
    y=df_combined['spd_10m_c'],
    mode='markers',
    marker=dict(
        size=12,
        # color by time of day
        color=df_combined.index.hour,
        # add colorscale
        colorscale='Viridis',
        showscale=True,
        line=dict(width=2, color='DarkSlateGrey')
    ),
    name='Wind Speed',
    
))
fig.update_layout(
    title='Wind Speed Aloft vs. Near Surface Using Dopple Lidar on Windy Days',
    xaxis_title='Winds at 1000m above surface',
    yaxis_title='Winds at 10m above surface',
    height=600,
    width=800,
    xaxis=dict(title_font=dict(size=18),),
    yaxis=dict(title_font=dict(size=18),),
)
# add time to hover
fig.update_traces(hovertemplate='Time: %{text}h<br>Wind Speed: %{x}m/s')